In [1]:
##### DNN module

from keras.utils import to_categorical
from keras import backend as K
from keras.models import Model
from keras.applications import ResNet50
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D

Using TensorFlow backend.


In [2]:
##### Common moldule

import numpy as np
import os
import glob
import matplotlib.pyplot as plt
import pandas as pd
from PIL import Image
from scipy.interpolate import RegularGridInterpolator
import time

In [3]:
##### Path

train_db = 'database/combined_train_mx_240.npy'
label_db = 'database/combined_label_mx_240.npy'

flower_li = list(map(os.path.basename, glob.glob('database/image_data/train/*')))
flower_mapping = {flower_li[i]: i for i in range(5)}

print(flower_mapping)

{'daisy': 0, 'dandelion': 1, 'rose': 2, 'sunflower': 3, 'tulip': 4}


In [4]:
##### Load data

flower_data   = np.load(train_db)
flower_target = np.load(label_db)

x_train, x_test, y_train, y_test = train_test_split(flower_data, flower_target, test_size=0.1, random_state=334)

In [5]:
##### Dimenssion check

(np.shape(x_train), np.shape(x_test), np.shape(y_train), np.shape(y_test))

((2540, 240, 240, 3), (283, 240, 240, 3), (2540,), (283,))

In [6]:
##### Normalize / Categorize

x_train = x_train / 255.
x_test = x_test / 255.
y_train = to_categorical(y_train, 5)
y_test = to_categorical(y_test, 5)

In [7]:
##### Parameter

image_size = (240, 240)
num_class = 5
batch_size = 16
num_epoches = 10
#freeze_layer = 2

In [8]:
net = ResNet50(include_top=False,
               weights='imagenet',
               input_tensor=None,
               input_shape=(image_size[0], image_size[1], 3))

C:\Users\Sauce\.conda\envs\megalopolis\lib\site-packages\keras_applications\resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


In [9]:
x = net.output
x = Flatten()(x)
x = Dropout(0.5)(x)
output_layer = Dense(num_class, activation='softmax', name='softmax')(x)

In [10]:
# 設定凍結與要進行訓練的網路層
net_final = Model(inputs=net.input, outputs=output_layer)
#for layer in net_final.layers[:freeze_layer]:
#    layer.trainable = False
#for layer in net_final.layers[freeze_layer:]:
#    layer.trainable = True

In [11]:
# 使用 Adam optimizer，以較低的 learning rate 進行 fine-tuning
net_final.compile(optimizer=Adam(lr=1e-5),
                  loss='categorical_crossentropy', metrics=['accuracy'])

In [12]:
# 輸出整個網路結構
print(net_final.summary())

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 240, 240, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 246, 246, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 120, 120, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 120, 120, 64) 256         conv1[0][0]                      
____________________________________________________________________________________________

In [13]:
history = net_final.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=num_epoches,
                    verbose=1,
                    validation_data=(x_test, y_test))

Train on 2540 samples, validate on 283 samples
Epoch 1/10
2540/2540 [==============================] - 104s 41ms/step - loss: 2.1794 - accuracy: 0.4272 - val_loss: 2.9760 - val_accuracy: 0.1237
Epoch 2/10
2540/2540 [==============================] - 89s 35ms/step - loss: 0.8495 - accuracy: 0.7378 - val_loss: 2.5848 - val_accuracy: 0.2226
Epoch 3/10
2528/2540 [============================>.] - ETA: 0s - loss: 0.4606 - accuracy: 0.8430

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\Sauce\.conda\envs\megalopolis\lib\site-packages\IPython\core\interactiveshell.py", line 3418, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-13-2ab91baf1a57>", line 5, in <module>
    validation_data=(x_test, y_test))
  File "C:\Users\Sauce\.conda\envs\megalopolis\lib\site-packages\keras\engine\training.py", line 1239, in fit
    validation_freq=validation_freq)
  File "C:\Users\Sauce\.conda\envs\megalopolis\lib\site-packages\keras\engine\training_arrays.py", line 210, in fit_loop
    verbose=0)
  File "C:\Users\Sauce\.conda\envs\megalopolis\lib\site-packages\keras\engine\training_arrays.py", line 449, in test_loop
    batch_outs = f(ins_batch)
  File "C:\Users\Sauce\.conda\envs\megalopolis\lib\site-packages\tensorflow_core\python\keras\backend.py", line 3727, in __call__
    outputs = self._graph_fn(*converted_inputs)
  File "C:\Users\Sauce\.conda\envs\megalopolis\lib\site-packages\tensorflow

TypeError: object of type 'NoneType' has no len()

In [14]:
score = net_final.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.47696288377573126
Test accuracy: 0.8303886651992798


In [15]:
net_final.save('ResNet50_model_2.h5')

In [16]:
##### Test on unknown sample

In [17]:
path_unknown = glob.glob('database/image_data/test/*')
path_unknown[:5]

['database/image_data/test\\0028624c49b3e0610ff9f1d111f5d532.jpg',
 'database/image_data/test\\002c30700185b7971369258b438070d5.jpg',
 'database/image_data/test\\00852f4f666acecd0c0d140365b42efd.jpg',
 'database/image_data/test\\00c08828fce04e360c732cac01edad9e.jpg',
 'database/image_data/test\\00d366e7877b6a78b104b57d67b60e6b.jpg']

In [18]:
def resampleRGI3d(input_mx, resize_to, dtype='float64'):
    # Ref: 10 3-A-7
    # Ver. 2.1
    # input_mx : numpy array, the original target matrix
    # resize_to: list or tuple with 3 int inside
    a, b, c = np.shape(input_mx)
    p, q, r = resize_to
    z_grid = np.linspace(0, p - 1, a)
    y_grid = np.linspace(0, q - 1, b)
    x_grid = np.linspace(0, r - 1, c)
    RGI = RegularGridInterpolator((z_grid, y_grid, x_grid), input_mx)
    z_grid_t2 = np.arange(p)
    y_grid_t2 = np.arange(q)
    x_grid_t2 = np.arange(r)
    meshgrid_para = np.meshgrid(z_grid_t2, y_grid_t2, x_grid_t2)
    RGI_mesh_mx = RGI((meshgrid_para[0], meshgrid_para[1], meshgrid_para[2]))
    RGI_mx = np.transpose(RGI_mesh_mx, axes=[1, 0, 2]).astype(dtype)
    return RGI_mx

def happy_time(start,stop):
    process_time = round(stop - start)
    ss = process_time % 60
    mm = process_time // 60 % 60
    hh = process_time // 3600
    duration = "Process time == {}s == {}H {}m {}s".format(process_time,hh,mm,ss)
    return duration

In [19]:
id_li = []
flower_class = []

loop_start = time.time()

for i in range(len(path_unknown)):
    f_id = os.path.basename(path_unknown[i])[:-4]
    temp_pic = np.asarray(Image.open(path_unknown[i]))
    temp_shape = np.shape(temp_pic)
    if temp_shape[0] > temp_shape[1]:
        temp_pic = temp_pic.transpose((1, 0, 2))
    temp_pic = resampleRGI3d(temp_pic, (180, 240, 3), dtype='uint8')
    temp_pic = np.expand_dims(temp_pic, axis = 0)
    pred = net_final.predict(temp_pic)[0]
    id_li.append(f_id)
    flower_class.append(list(pred).index(max(pred)))
    if (i+1) % 200 == 0:
        print(i+1, 'files done.', happy_time(loop_start, time.time()))

200 files done. Process time == 11s == 0H 0m 11s
400 files done. Process time == 20s == 0H 0m 20s
600 files done. Process time == 28s == 0H 0m 28s
800 files done. Process time == 37s == 0H 0m 37s
1000 files done. Process time == 46s == 0H 0m 46s
1200 files done. Process time == 54s == 0H 0m 54s
1400 files done. Process time == 63s == 0H 1m 3s
1600 files done. Process time == 71s == 0H 1m 11s
1800 files done. Process time == 80s == 0H 1m 20s
2000 files done. Process time == 88s == 0H 1m 28s


In [20]:
pred_result_df = pd.DataFrame(columns=['id', 'flower_class'])
pred_result_df['id'] = id_li
pred_result_df['flower_class'] = flower_class
pred_result_df

,id,flower_class
0,0028624c49b3e0610ff9f1d111f5d532,3
1,002c30700185b7971369258b438070d5,4
2,00852f4f666acecd0c0d140365b42efd,3
3,00c08828fce04e360c732cac01edad9e,1
4,00d366e7877b6a78b104b57d67b60e6b,2
...,...,...
1995,ff7eac29b6d7a33fbd8009677c3e9c58,3
1996,ffbc32a7b67dfe72b8d35d4b1b35fd6c,4
1997,ffea1f275c05accb0a6bfd1203620c7e,1
1998,ffeb2a1cf53464b6af937ab8af0c2946,3


In [21]:
pred_result_df.to_csv('pred_result_resnet50_2.csv', index=False)